In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
from tqdm import tqdm

In [2]:
data=[]
for pg in tqdm(range(0,40)):
    link='https://indiankanoon.org/search/?formInput=property%20fraud%20%20%20%20%20doctypes%3A%20karnataka%20sortby%3A%20leastrecent&pagenum='+str(pg)
    res= requests.get(link)
    soup = BeautifulSoup(res.text,'html.parser')

    for sp in soup.find_all('div', class_='result'):
        title= sp.find('div',class_='result_title').text[1:-1]
        headline= sp.find('div',class_='headline').text[1:-1]
        detail_id=sp.find('a').get('href')
        data.append([title,headline,detail_id])
print(data)
df=pd.DataFrame(data, columns = ['title','headline','detail_id'])
df.to_csv('Datasetleastrecent.csv',index=False)


100%|██████████| 40/40 [00:21<00:00,  1.86it/s]

[['In Re: Supreme Bank Of India Ltd. (In ... vs Unknown on 1 January, 1800', '     In Re: Supreme Bank Of India Ltd. (In ... vs Unknown on 1 January, 1800 \n\n JUDGMENT \n ', '/docfragment/894782/?formInput=property%20fraud%20%20%20%20%20%20doctypes%3A%20karnataka%20sortby%3A%20leastrecent'], ['Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs R. Vishwanathan And Ors. on 29 July, 1949', '     Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs R. Vishwanathan And Ors. on 29 \n ', '/docfragment/1914739/?formInput=property%20fraud%20%20%20%20%20%20doctypes%3A%20karnataka%20sortby%3A%20leastrecent'], ['Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs Gajambal Ramalingam And Ors. on 6 January, 1950', '     Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs Gajambal Ramalingam And Ors. on 6 \n ', '/docfragment/1812794/?formInput=property%20fraud%20%20%20%20%20%20doctypes%3A%20karnataka%20sortby%3A%20leastrecent'], ['Abdul Wahab And Ors. vs Ramakrishniah on 31 March, 1950', '     Abdul Wahab And Ors. vs Ramakrishniah on 31 March

In [3]:
df.shape


(400, 3)

In [4]:
import re
def function(text):
    # Find the index of '?', if present
    index = text.find('?')
    if index != -1:
        return text[:index] + '/'  # Return the substring up to '?' plus '/'
    return text  # Return the whole text if '?' is not found

# Apply the function to the 'Case Link' column
df["new_link"] = df["detail_id"].apply(function)
df['new_link'] = 'https://indiankanoon.org' + df['new_link']
# Display the modified DataFrame
df.head()

,title,headline,detail_id,new_link
0,In Re: Supreme Bank Of India Ltd. (In ... vs U...,In Re: Supreme Bank Of India Ltd. (In ......,/docfragment/894782/?formInput=property%20frau...,https://indiankanoon.org/docfragment/894782//
1,Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs R. V...,Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs...,/docfragment/1914739/?formInput=property%20fra...,https://indiankanoon.org/docfragment/1914739//
2,Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs Gajamb...,Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs G...,/docfragment/1812794/?formInput=property%20fra...,https://indiankanoon.org/docfragment/1812794//
3,Abdul Wahab And Ors. vs Ramakrishniah on 31 Ma...,Abdul Wahab And Ors. vs Ramakrishniah on ...,/docfragment/1246365/?formInput=property%20fra...,https://indiankanoon.org/docfragment/1246365//
4,K.V. Bhaskara Rao And Anr. vs S. Naranappa on ...,"family, lands. In execution of that decree...",/docfragment/348382/?formInput=property%20frau...,https://indiankanoon.org/docfragment/348382//


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

# Base URL for the search is already done and saved in your DataFrame df
# Assuming you have a DataFrame with "Case Title" and "Case Link" columns

# List to store case details
case_details_list = []

# Loop through each row in the DataFrame to extract case details
for index, row in tqdm(df.iterrows()):
    case_link = row['new_link']
    
    # Send a GET request to the case link
    response = requests.get(case_link)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the case document's HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the relevant case details from the document (you may need to adjust the selector)
        # Look for the main content where the case details are stored
        case_details_div = soup.find('div', class_='judgments')  # Example class for the document content
        
        if case_details_div:
            case_details = case_details_div.get_text(separator="\n").strip()  # Get the full document text
        else:
            case_details = "No case details found."
    else:
        case_details = "Failed to retrieve case details."

    # Append the case details to the list
    case_details_list.append(case_details)
    
    # Adding a small delay to avoid overwhelming the server
    time.sleep(1)  # Sleep for 1 second between requests

# Add the case details to the DataFrame
df['Case Details'] = case_details_list

# Display the updated DataFrame
print(df)

# Optionally, save it to a CSV file with case titles, links, and case details
df.to_csv('caseleastleastrecent.csv', index=False)

df.head()

400it [11:03,  1.66s/it]


                                                 title  \
0    In Re: Supreme Bank Of India Ltd. (In ... vs U...   
1    Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs R. V...   
2    Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs Gajamb...   
3    Abdul Wahab And Ors. vs Ramakrishniah on 31 Ma...   
4    K.V. Bhaskara Rao And Anr. vs S. Naranappa on ...   
..                                                 ...   
395  K.P. Krishnakumar vs Smt. Radhalakshmi Amma on...   
396  Syndicate Bank vs Mahalaxmi Ginning Factory An...   
397  B. Bennari vs State Of Karnataka By Its Secret...   
398  Sri Parushuram Nemani Kuduchakar And ... vs Sm...   
399  Soore Detergents, By Proprietor vs D. Mohanraj...   

                                              headline  \
0         In Re: Supreme Bank Of India Ltd. (In ......   
1         Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs...   
2         Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs G...   
3         Abdul Wahab And Ors. vs Ramakrishniah on ...   
4        fami

,title,headline,detail_id,new_link,Case Details
0,In Re: Supreme Bank Of India Ltd. (In ... vs U...,In Re: Supreme Bank Of India Ltd. (In ......,/docfragment/894782/?formInput=property%20frau...,https://indiankanoon.org/docfragment/894782//,Take notes as you read a judgment using our \n...
1,Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs R. V...,Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs...,/docfragment/1914739/?formInput=property%20fra...,https://indiankanoon.org/docfragment/1914739//,Take notes as you read a judgment using our \n...
2,Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs Gajamb...,Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs G...,/docfragment/1812794/?formInput=property%20fra...,https://indiankanoon.org/docfragment/1812794//,Take notes as you read a judgment using our \n...
3,Abdul Wahab And Ors. vs Ramakrishniah on 31 Ma...,Abdul Wahab And Ors. vs Ramakrishniah on ...,/docfragment/1246365/?formInput=property%20fra...,https://indiankanoon.org/docfragment/1246365//,Take notes as you read a judgment using our \n...
4,K.V. Bhaskara Rao And Anr. vs S. Naranappa on ...,"family, lands. In execution of that decree...",/docfragment/348382/?formInput=property%20frau...,https://indiankanoon.org/docfragment/348382//,Take notes as you read a judgment using our \n...


In [6]:
df['Case Details'][0]

'Take notes as you read a judgment using our \nVirtual Legal Assistant\n and get email alerts whenever a new judgment matches your query (\nQuery Alert Service\n). Try out our \nPremium Member Services\n -- \nSign up today\n and get free trial for one month.\n\n\n\n\nKarnataka High Court\n\n\nIn Re: Supreme Bank Of India Ltd. (In ... vs Unknown on 1 January, 1800\n\n\nJUDGMENT\n\n\n 1. This is an application by the official liquidator under \nsection 235\n of the Indian Companies Act of 1913 (Central Act VII of 1913) read with \nsection 45H\n of the Banking Companies Act of 1949 to examine into the conduct of fourteen persons impleaded as respondents, 7 of whom were directors, one the auditor and the rest employees of the company named above, in relation to the affairs of the company, and to direct them either jointly and severally or in such manner as the court may deem just to repay or restore to the company a sum of Rs. 4,26,000 said to have been misapplied or retained by the respon

In [7]:
import re

# Function to clean and format case details
def clean_case_details(text):
    # Replace excessive newlines and spaces
    cleaned_text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with a single newline
    cleaned_text = re.sub(r' {2,}', ' ', cleaned_text)  # Replace multiple spaces with a single space

    # Replace abbreviations like \nMACT\n with proper format
    cleaned_text = cleaned_text.replace('\nMACT\n', ' MACT ')
    cleaned_text = cleaned_text.replace('\nMotor Vehicles Act, 1988\n', ' Motor Vehicles Act, 1988 ')
    cleaned_text = cleaned_text.replace('\nArticle 226\n', ' Article 226 ')
    
    # Further format or replace other commonly occurring text, if necessary

    # Trim any leading/trailing whitespace
    cleaned_text = cleaned_text.strip()
    
    
    return cleaned_text

# Apply the cleaning function to all rows in the 'Case Details' column
df['Case Details'] = df['Case Details'].apply(clean_case_details)
df['Case Details'] = df['Case Details'].str.replace('\t', ' ')
df['Case Details'] = df['Case Details'].str.replace('\n', ' ')
df['Case Details'] = df['Case Details'].str.replace("\'", ' ')
# Display the cleaned DataFrame
print(df[['title', 'Case Details']].head())

# Optionally save the cleaned data to a CSV
df.to_csv('cleaned_case_details_leaseleastrecent.csv', index=False)      

                                               title  \
0  In Re: Supreme Bank Of India Ltd. (In ... vs U...   
1  Rukn-Ul-Mulk Syed Abdul Wajid And Ors. vs R. V...   
2  Rukn-Ul-Mulk S. Abdul Wajid And Ors. vs Gajamb...   
3  Abdul Wahab And Ors. vs Ramakrishniah on 31 Ma...   
4  K.V. Bhaskara Rao And Anr. vs S. Naranappa on ...   

                                        Case Details  
0  Take notes as you read a judgment using our  V...  
1  Take notes as you read a judgment using our  V...  
2  Take notes as you read a judgment using our  V...  
3  Take notes as you read a judgment using our  V...  
4  Take notes as you read a judgment using our  V...  


In [8]:
df['Case Details'][0]

'Take notes as you read a judgment using our  Virtual Legal Assistant  and get email alerts whenever a new judgment matches your query ( Query Alert Service ). Try out our  Premium Member Services  --  Sign up today  and get free trial for one month. Karnataka High Court In Re: Supreme Bank Of India Ltd. (In ... vs Unknown on 1 January, 1800 JUDGMENT  1. This is an application by the official liquidator under  section 235  of the Indian Companies Act of 1913 (Central Act VII of 1913) read with  section 45H  of the Banking Companies Act of 1949 to examine into the conduct of fourteen persons impleaded as respondents, 7 of whom were directors, one the auditor and the rest employees of the company named above, in relation to the affairs of the company, and to direct them either jointly and severally or in such manner as the court may deem just to repay or restore to the company a sum of Rs. 4,26,000 said to have been misapplied or retained by the respondents or in respect of which they ar